In [8]:
import timm
import torch
import torch.nn as nn
import mlflow
import mlflow.pytorch

In [14]:
import os
import sys

# 추가하고자 하는 경로
directory_to_add = "/home/work/Deeplant-AI/Deeplant-AI-main"

# 절대 경로로 변환
absolute_path = os.path.abspath(directory_to_add)

# sys.path에 경로 추가
if absolute_path not in sys.path:
    sys.path.append(directory_to_add)

print(f"Added to sys.path: {absolute_path}")


Added to sys.path: /home/work/Deeplant-AI/Deeplant-AI-main


In [15]:
mlflow.set_tracking_uri('http://0.0.0.0:5000')
model_uri = "models:/resnetv2.101x3/2"

model = mlflow.pytorch.load_model(model_uri)

In [16]:
print(model)

MLP_layer(
  (base_model): BaseModel(
    (base_model): ResNetV2(
      (stem): Sequential(
        (conv): StdConv2d(3, 192, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (pad): ConstantPad2d(padding=(1, 1, 1, 1), value=0.0)
        (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (stages): Sequential(
        (0): ResNetStage(
          (blocks): Sequential(
            (0): PreActBottleneck(
              (downsample): DownsampleConv(
                (conv): StdConv2d(192, 768, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (norm): Identity()
              )
              (norm1): GroupNormAct(
                32, 192, eps=1e-05, affine=True
                (drop): Identity()
                (act): ReLU(inplace=True)
              )
              (conv1): StdConv2d(192, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (norm2): GroupNormAct(
                32, 192, eps=1e-05, af

In [1]:
import timm
import torch
import torch.nn as nn

class BaseModel(nn.Module):
    def __init__(self, model_name, pretrained, num_classes, in_chans):
        super(BaseModel, self).__init__()
        self.base_model = timm.create_model(model_name=model_name, pretrained=pretrained, num_classes=num_classes, in_chans=in_chans)
        
        #self.base_model.conv_head = nn.Identity()
        #self.base_model.norm_head = nn.Identity()
        
    def forward(self, x):
        x = self.base_model(x)
        return x

    

class MLP_layer(nn.Module):
    def __init__(self, base_model, out_dim):
        super().__init__()
        self.base_model = base_model
        self.out_dim = out_dim

        self.num_features = self.base_model.base_model.num_features
        
        # out_dim 개수만큼 MLP 생성
        self.mlp_heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(self.num_features, 64),
                nn.ReLU(),
                nn.Linear(64, 1)
            ) for _ in range(out_dim)
        ])

    def forward(self, x):
        print(x.shape)
        base_output = self.base_model(x)

        print(self.num_features)
        print(base_output.shape)
        
        features = base_output

        outputs = [head(features) for head in self.mlp_heads]

        return torch.cat(outputs, dim=1)

def create_model(model_name, pretrained, num_classes, in_chans, out_dim):

    if out_dim <= 0:
        raise ValueError("오류: out_dim이 0 이하입니다.")

    base_model = BaseModel(model_name, pretrained, num_classes, in_chans)
    model = MLP_layer(base_model, out_dim)

    return model


In [2]:
model = create_model("swin_tiny_patch4_window7_224", True, 5, 3, 5)

print(model)

with torch.no_grad():
    sample_input = torch.randn(5, 3, 224, 224)  # 일반적인 입력 크기
    base_output = model(sample_input)
    num_features = base_output.shape[1]  # 출력 크기의 두 번째 차원이 feature 크기
    print(num_features)

model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

MLP_layer(
  (base_model): BaseModel(
    (base_model): SwinTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
        (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      )
      (layers): Sequential(
        (0): SwinTransformerStage(
          (downsample): Identity()
          (blocks): Sequential(
            (0): SwinTransformerBlock(
              (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
              (attn): WindowAttention(
                (qkv): Linear(in_features=96, out_features=288, bias=True)
                (attn_drop): Dropout(p=0.0, inplace=False)
                (proj): Linear(in_features=96, out_features=96, bias=True)
                (proj_drop): Dropout(p=0.0, inplace=False)
                (softmax): Softmax(dim=-1)
              )
              (drop_path1): Identity()
              (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
              (mlp): Mlp(


768
torch.Size([5, 5])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (5x5 and 768x64)

In [4]:
def conv3x3(in_dim, out_dim):
    return torch.nn.Sequential(
        nn.Conv2d(in_dim, out_dim, kernel_size=3, stride=2, padding=1),
        nn.BatchNorm2d(out_dim)
    )

class Affine(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.alpha = nn.Parameter(torch.ones([1, dim, 1, 1]))
        self.beta = nn.Parameter(torch.zeros([1, dim, 1, 1]))

    def forward(self, x):
        return x * self.alpha + self.beta

class SOPE(nn.Module):
    def __init__(self, patch_size, embed_dim):
        super().__init__()
        self.pre_affine = Affine(3)
        self.post_affine = Affine(embed_dim)

        if patch_size[0] == 16:
            self.proj = torch.nn.Sequential(
                conv3x3(3, embed_dim // 8),
                nn.GELU(),
                conv3x3(embed_dim // 8, embed_dim // 4),
                nn.GELU(),
                conv3x3(embed_dim // 4, embed_dim // 2),
                nn.GELU(),
                conv3x3(embed_dim // 2, embed_dim),
            )
        elif patch_size[0] == 4:
            self.proj = torch.nn.Sequential(
                conv3x3(3, embed_dim // 2),
                nn.GELU(),
                conv3x3(embed_dim // 2, embed_dim),
            )
        elif patch_size[0] == 2:
            self.proj = torch.nn.Sequential(
                conv3x3(3, embed_dim),
                nn.GELU(),
            )

    def forward(self, x):
        B, C, H, W = x.shape
        x = self.pre_affine(x)
        x = self.proj(x)
        x = self.post_affine(x)
        Hp, Wp = x.shape[2], x.shape[3]
        x = x.flatten(2).transpose(1, 2)
        return x


class DAFF(nn.Module):
    def __init__(self, in_dim, hid_dim, out_dim, kernel_size=3):
        super().__init__()
        self.conv1 = nn.Conv2d(in_dim, hid_dim, kernel_size=1, stride=1, padding=0)
        self.conv2 = nn.Conv2d(hid_dim, hid_dim, kernel_size=3, stride=1, padding=(kernel_size-1)//2, groups=hid_dim)
        self.conv3 = nn.Conv2d(hid_dim, out_dim, kernel_size=1, stride=1, padding=0)
        self.act = nn.GELU()
        self.squeeze = nn.AdaptiveAvgPool2d((1, 1))
        self.compress = nn.Linear(in_dim, in_dim // 4)
        self.excitation = nn.Linear(in_dim // 4, in_dim)
        self.bn1 = nn.BatchNorm2d(hid_dim)
        self.bn2 = nn.BatchNorm2d(hid_dim)
        self.bn3 = nn.BatchNorm2d(out_dim)

    def forward(self, x):
        B, N, C = x.size()
        cls_token, tokens = torch.split(x, [1, N-1], dim=1)
        x = tokens.reshape(B, int(math.sqrt(N-1)), int(math.sqrt(N-1)), C).permute(0, 3, 1, 2)
        x = self.act(self.bn1(self.conv1(x)))
        x = x + self.act(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        weight = self.squeeze(x).flatten(1).reshape(B, 1, C)
        weight = self.excitation(self.act(self.compress(weight)))
        cls_token = cls_token * weight
        tokens = x.flatten(2).permute(0, 2, 1)
        out = torch.cat((cls_token, tokens), dim=1)
        return out
    
class SelectAdaptivePool2d(nn.Module):
    def __init__(self, pool_type='avg', flatten=True):
        super(SelectAdaptivePool2d, self).__init__()
        if pool_type == 'avg':
            self.pool = nn.AdaptiveAvgPool2d((1, 1))  # Adaptive Average Pooling
        elif pool_type == 'max':
            self.pool = nn.AdaptiveMaxPool2d((1, 1))  # Adaptive Max Pooling
        else:
            raise ValueError("Unsupported pool_type: {}".format(pool_type))
        
        self.flatten = flatten

    def forward(self, x):
        # x shape: (batch_size, 7, 7, 768)
        x = x.permute(0, 3, 1, 2)  # Change to (batch_size, 768, 7, 7)
        x = self.pool(x)  # Pooling to (batch_size, 768, 1, 1)
        if self.flatten:
            x = x.view(x.size(0), -1)  # Flatten to (batch_size, 768)
        return x 

class BaseModel(nn.Module):
    def __init__(self, model_name, pretrained, num_classes, in_chans, embed_layer=SOPE):
        super(BaseModel, self).__init__()
        self.base_model = timm.create_model(model_name=model_name, pretrained=pretrained, num_classes=num_classes, in_chans=in_chans)
        
        self.base_model.head.global_pool = nn.Identity()
        self.global_pool = SelectAdaptivePool2d()
        
#         self.base_model.patch_embed = SOPE(patch_size=(16, 16), embed_dim=768)
#         for i in range(len(self.base_model.blocks)):
#             block = self.base_model.blocks[i]
#             in_features = block.mlp.fc1.in_features
#             hidden_features = block.mlp.fc1.out_features
#             out_features = block.mlp.fc2.out_features

#             # DAFF 모듈로 교체
#             block.mlp = DAFF(in_features, hidden_features, out_features)
        
    def forward(self, x):
        # 패치 임베딩
#         x = self.base_model.patch_embed(x)
#         print(f"After patch embedding: {x.shape}")  # (batch_size, num_patches, embed_dim)

#         # 클래스 토큰 추가
#         batch_size = x.shape[0]
#         print("before:", self.base_model.cls_token.shape)
#         cls_token = self.base_model.cls_token.expand(batch_size, -1, -1)  # (batch_size, 1, embed_dim)
#         print("after:", cls_token.shape)
#         x = torch.cat((cls_token, x), dim=1)  # (batch_size, num_patches + 1, embed_dim)
#         print(f"After adding class token: {x.shape}")
        
#         # 포지셔널 임베딩 추가
#         x = x + self.base_model.pos_embed  # (batch_size, num_patches + 1, embed_dim)
#         x = self.base_model.pos_drop(x)
#         print(f"After positional embedding: {x.shape}")
        
#         # 트랜스포머 블록 통과
#         x = self.base_model.blocks(x)
#         print(f"After transformer blocks: {x.shape}")
        
#         # LayerNorm 적용
#         x = self.base_model.norm(x)
#         print(f"After norm: {x.shape}")
        
#         # Class token 선택
#         cls_token_final = x[:, 0]
#         print(f"After selecting class token: {cls_token_final.shape}")
        
#         # Classifier head 적용
#         x = self.base_model.head(cls_token_final)
#         print(f"Final output shape: {x.shape}")

        x = self.base_model.patch_embed(x)
        print(f"After patch embedding: {x.shape}")  # (batch_size, num_patches, embed_dim)
        
        x = self.base_model.layers(x)
        print(x.shape)
        
        x = self.base_model.norm(x)
        print(x.shape)
        
        print("before global pool:", x.shape)
        x = self.global_pool(x)
        print("after:", x.shape)

        return x

In [5]:
class MLP_layer(nn.Module):
    def __init__(self, base_model, out_dim):
        super().__init__()
        self.base_model = base_model
        self.out_dim = out_dim

        self.num_features = self.base_model.base_model.num_features
        
        # out_dim 개수만큼 MLP 생성
        self.mlp_heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(self.num_features, 64),
                nn.ReLU(),
                nn.Linear(64, 1)
            ) for _ in range(out_dim)
        ])

    def forward(self, x):
        base_output = self.base_model(x)
        
        print("mlp input:", base_output.shape)
        
        features = base_output

        outputs = [head(features) for head in self.mlp_heads]

        return torch.cat(outputs, dim=1)

In [6]:
def create_model(model_name, pretrained, num_classes, in_chans, out_dim):

    if out_dim <= 0:
        raise ValueError("오류: out_dim이 0 이하입니다.")

    base_model = BaseModel(model_name, pretrained, num_classes, in_chans)
    model = MLP_layer(base_model, out_dim)

    return model

In [7]:
model_name = 'swin_s3_base_224'
pretrained = True
num_classes = 0
in_chans = 3
out_dim = 5

model = create_model(model_name, pretrained, num_classes, in_chans, out_dim)

print(model)

model.safetensors:   0%|          | 0.00/295M [00:00<?, ?B/s]

MLP_layer(
  (base_model): BaseModel(
    (base_model): SwinTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
        (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      )
      (layers): Sequential(
        (0): SwinTransformerStage(
          (downsample): Identity()
          (blocks): Sequential(
            (0): SwinTransformerBlock(
              (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
              (attn): WindowAttention(
                (qkv): Linear(in_features=96, out_features=288, bias=True)
                (attn_drop): Dropout(p=0.0, inplace=False)
                (proj): Linear(in_features=96, out_features=96, bias=True)
                (proj_drop): Dropout(p=0.0, inplace=False)
                (softmax): Softmax(dim=-1)
              )
              (drop_path1): Identity()
              (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
              (mlp): Mlp(


In [22]:
#custom model

import timm
import torch
import torch.nn as nn
import torch.nn.functional as F

# !export PYTHONPATH=$PYTHONPATH:/home/work/tiny-transformers/tiny-transformers

# +
# from pycls.models.cnns.resnet import ResNet  # resnet.py에서 ResNet 모델 불러오기

class TeacherModel(nn.Module):
    def __init__(self, model_name='vit_base_r50_s16_224.orig_in21k', pretrained=True, num_classes=0, load_from_file=False, model_path=None):
        super(TeacherModel, self).__init__()
        self.model = None
        self.features = []
        self.feature_layers = ['patch_embed', 'blocks.5', 'blocks.10', 'norm']  # Hook after key transformer blocks

        if load_from_file and model_path:
            # Load model from a saved .pth file
            self._load_pretrained_model(model_path)
        else:
            # Use timm to create a model
            self.model = timm.create_model(model_name=model_name, pretrained=pretrained, num_classes=num_classes)
        
        self.model.eval()
        self._register_hooks()

    def _load_pretrained_model(self, model_path):
        # Load the saved model from the provided path
        print(f"Loading teacher model from {model_path}")
        checkpoint = torch.load(model_path, map_location=torch.device('cpu'))
        # self.model.load_state_dict(checkpoint, strict=False)
        # self.model = nn.Sequential(*list(checkpoint['model'].children())[:-1])  # Assuming last two layers are not required
        # self.model.load_state_dict(checkpoint['state_dict'])
        # Check if the checkpoint is a full model or just a state dict
        if 'state_dict' in checkpoint:
            state_dict = checkpoint['state_dict']  # If saved as a state dict
        else:
            state_dict = checkpoint  # If it's directly the state dict

        # Initialize timm model
        self.model = timm.create_model(model_name='vit_base_r50_s16_224.orig_in21k', pretrained=False, num_classes=0)
        
        # Adjust the state dict keys if necessary (remove 'base_model.base_model.')
        new_state_dict = {}
        for key in state_dict.keys():
            new_key = key.replace('base_model.base_model.', 'model.')  # Adjust the prefix
            new_state_dict[new_key] = state_dict[key]

        # Load state dict into model
        self.model.load_state_dict(new_state_dict, strict=False)
        print("Teacher model loaded successfully!")

    def _register_hooks(self):
        def hook(module, input, output):
            self.features.append(output)

        for name, module in self.model.named_modules():
            if name in self.feature_layers:
                module.register_forward_hook(hook)

    def forward(self, x):
        self.features = []
        output = self.model(x)
        return output, self.features


class StudentModel(nn.Module):
    def __init__(self, model_name, pretrained, num_classes, in_chans):
        super(StudentModel, self).__init__()
        self.base_model = timm.create_model(model_name=model_name, pretrained=pretrained, num_classes=num_classes, in_chans=in_chans)
        self.features = []
        self.feature_layers = ['layers.0', 'layers.1', 'layers.2', 'layers.3']  # Hook SwinTransformerStage blocks
        self._register_hooks()

    def _register_hooks(self):
        def hook(module, input, output):
            self.features.append(output)

        
        for name, module in self.base_model.named_modules():
            if name in self.feature_layers:
                module.register_forward_hook(hook)


    def forward(self, x):
        self.features = []
        output = self.base_model(x)
        return output, self.features

class FeatureTransform(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(FeatureTransform, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)
    
    def forward(self, x):
        # 입력이 [B, H, W, C] 형태라면 [B, C, H, W]로 변환
        if x.dim() == 4 and x.shape[1] != x.shape[-1]:
            x = x.permute(0, 3, 1, 2)
        
        x = self.conv(x)
        
        
        return x


class DistillationModel(nn.Module):
    def __init__(self, teacher_model, student_model):
        super(DistillationModel, self).__init__()
        self.teacher_model = teacher_model
        self.student_model = student_model
        self.output_transform = nn.Linear(768, 5)
        self.transforms = None  # 초기에는 None으로 설정

    def forward(self, x):
        # print("DistillationModel forward pass started", flush=True)

        device = x.device  # Get the device of the input tensor

        with torch.no_grad():
            teacher_output, teacher_features = self.teacher_model(x)  # Teacher features remain frozen.
    
        student_output, student_features = self.student_model(x)  # Student features should be updated.

        # transforms가 아직 초기화되지 않았다면 여기서 초기화
        if self.transforms is None:
            self.transforms = nn.ModuleList()
            for t_feat, s_feat in zip(teacher_features, student_features[:-1]):
                in_channels = s_feat.shape[-1]  # Student feature 채널 수
                out_channels = t_feat.shape[1]  # Teacher feature 채널 수
                transform = FeatureTransform(in_channels, out_channels)
                self.transforms.append(transform.to(device))  # Move the transform to the correct device


        # 손실 및 변환을 동시에 계산
        distillation_loss = torch.tensor(0.0, device=device, requires_grad=True)  # 초기화
        for i, (transform, s_feat, t_feat) in enumerate(zip(self.transforms, student_features[:-1], teacher_features)):
            # print(f"Layer {i+1} - Before Transformation - Student feature shape: {s_feat.shape}")
            # print(f"Before Conv: {s_feat.shape}", flush=True)
            
            # 학생 특징 변환
            transformed_feat = transform(s_feat)  # permute 필요 없음
            # print(f"Layer {i+1} - After Transformation - Transformed feat shape: {transformed_feat.shape}", flush=True)

            # 손실 계산
            layer_loss = F.mse_loss(transformed_feat, t_feat)
            distillation_loss = distillation_loss + layer_loss
            # print(f"Layer {i+1} loss: {layer_loss.item()}", flush=True)

        student_output_resized = self.output_transform(student_output)
        return student_output_resized, teacher_output, distillation_loss



# # MLP Layer to output 'out_dim' different outputs
# class MLP_layer(nn.Module):
#     def __init__(self, base_model, out_dim):
#         super().__init__()
#         self.base_model = base_model
#         self.out_dim = out_dim

#         # Get the number of features from the base model
#         self.num_features = self.base_model.base_model.num_features

#         # Create MLP heads, one for each output dimension (out_dim = 5 in this case)
#         self.mlp_heads = nn.ModuleList([
#             nn.Sequential(
#                 nn.Linear(self.num_features, 64),
#                 nn.ReLU(),
#                 nn.Linear(64, 1)
#             ) for _ in range(out_dim)
#         ])

#     def forward(self, x):
#         base_output = self.base_model(x)
#         features = base_output

#         # Apply each MLP head to the base model output
#         outputs = [head(features) for head in self.mlp_heads]

#         # Concatenate the outputs along the last dimension
#         return torch.cat(outputs, dim=1)

def create_model(student_model_name, teacher_model_name, pretrained, num_classes, in_chans, out_dim, load_from_file=False, model_path=None):
    teacher_model = TeacherModel(model_name=teacher_model_name, pretrained=pretrained, num_classes=num_classes, load_from_file=load_from_file, model_path=model_path)
    student_model = StudentModel(model_name=student_model_name, pretrained=pretrained, num_classes=num_classes, in_chans=in_chans)
    model = DistillationModel(teacher_model, student_model)

    return model


In [43]:
#custom model

import timm
import torch
import torch.nn as nn
import torch.nn.functional as F

# !export PYTHONPATH=$PYTHONPATH:/home/work/tiny-transformers/tiny-transformers

# +
# from pycls.models.cnns.resnet import ResNet  # resnet.py에서 ResNet 모델 불러오기

class TeacherModel(nn.Module):
    def __init__(self, model_name='resnet50', pretrained=True, num_classes=0, load_from_file=False, model_path=None):
        super(TeacherModel, self).__init__()
        self.model = None
        self.features = []
        self.feature_layers = ['layer1', 'layer2', 'layer3']

        if load_from_file and model_path:
            # Load model from a saved .pth file
            self._load_pretrained_model(model_path)
        else:
            # Use timm to create a model
            self.model = timm.create_model(model_name=model_name, pretrained=pretrained, num_classes=num_classes)
        
        self.model.eval()
        self._register_hooks()

    def _load_pretrained_model(self, model_path):
        # Load the saved model from the provided path
        print(f"Loading teacher model from {model_path}")
        checkpoint = torch.load(model_path, map_location=torch.device('cpu'))
        # self.model.load_state_dict(checkpoint, strict=False)
        # self.model = nn.Sequential(*list(checkpoint['model'].children())[:-1])  # Assuming last two layers are not required
        # self.model.load_state_dict(checkpoint['state_dict'])
        # Check if the checkpoint is a full model or just a state dict
        if 'state_dict' in checkpoint:
            state_dict = checkpoint['state_dict']  # If saved as a state dict
        else:
            state_dict = checkpoint  # If it's directly the state dict

        # Initialize timm model
        self.model = timm.create_model(model_name='resnet50', pretrained=False, num_classes=0)
        
        # Adjust the state dict keys if necessary (remove 'base_model.base_model.')
        new_state_dict = {}
        for key in state_dict.keys():
            new_key = key.replace('base_model.base_model.', 'model.')  # Adjust the prefix
            new_state_dict[new_key] = state_dict[key]

        # Load state dict into model
        self.model.load_state_dict(new_state_dict, strict=False)
        print("Teacher model loaded successfully!")

    def _register_hooks(self):
        def hook(module, input, output):
            self.features.append(output)

        for name, module in self.model.named_modules():
            if name in self.feature_layers:
                module.register_forward_hook(hook)

    def forward(self, x):
        self.features = []
        output = self.model(x)
        return output, self.features


class StudentModel(nn.Module):
    def __init__(self, model_name, pretrained, num_classes, in_chans):
        super(StudentModel, self).__init__()
        self.base_model = timm.create_model(model_name=model_name, pretrained=pretrained, num_classes=num_classes, in_chans=in_chans)
        self.features = []
        self.feature_layers = ['layers.0', 'layers.1', 'layers.2', 'layers.3']  # Hook SwinTransformerStage blocks
        self._register_hooks()

    def _register_hooks(self):
        def hook(module, input, output):
            self.features.append(output)

        
        for name, module in self.base_model.named_modules():
            if name in self.feature_layers:
                module.register_forward_hook(hook)


    def forward(self, x):
        self.features = []
        output = self.base_model(x)
        return output, self.features

class FeatureTransform(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(FeatureTransform, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)
    
    def forward(self, x):
        # 입력이 [B, H, W, C] 형태라면 [B, C, H, W]로 변환
        if x.dim() == 4 and x.shape[1] != x.shape[-1]:
            x = x.permute(0, 3, 1, 2)
        
        x = self.conv(x)
        
        
        return x



class DistillationModel(nn.Module):
    def __init__(self, teacher_model, student_model, beta=1.0):
        super(DistillationModel, self).__init__()
        self.teacher_model = teacher_model
        self.student_model = student_model
        self.output_transform = nn.Linear(768, 5)
        self.transforms = None
        self.beta = beta  # beta 값을 클래스 인스턴스 변수로 저장

    def forward(self, x, labels=None):
        device = x.device

        # Teacher 모델에서 feature 추출
        with torch.no_grad():
            teacher_output, teacher_features = self.teacher_model(x)

        # Student 모델에서 feature 추출
        student_output, student_features = self.student_model(x)

        # Feature Alignment
        if self.transforms is None:
            self.transforms = nn.ModuleList([
                FeatureTransform(s_feat.shape[-1], t_feat.shape[1]).to(device)
                for t_feat, s_feat in zip(teacher_features, student_features[:-1])
            ])

        # Distillation loss (guidance loss) 계산
        distillation_loss = torch.tensor(0.0, device=device)
        for transform, s_feat, t_feat in zip(self.transforms, student_features[:-1], teacher_features):
            s_feat_resized = transform(s_feat)
            distillation_loss += F.mse_loss(s_feat_resized, t_feat)

        # Classification Loss 계산
        student_output_resized = self.output_transform(student_output)
        classification_loss = F.cross_entropy(student_output_resized, labels) if labels is not None else torch.tensor(0.0, device=device)

        # Total Loss 계산
        total_loss = classification_loss + self.beta * distillation_loss
        return student_output_resized, teacher_output, classification_loss, distillation_loss, total_loss


# # MLP Layer to output 'out_dim' different outputs
# class MLP_layer(nn.Module):
#     def __init__(self, base_model, out_dim):
#         super().__init__()
#         self.base_model = base_model
#         self.out_dim = out_dim

#         # Get the number of features from the base model
#         self.num_features = self.base_model.base_model.num_features

#         # Create MLP heads, one for each output dimension (out_dim = 5 in this case)
#         self.mlp_heads = nn.ModuleList([
#             nn.Sequential(
#                 nn.Linear(self.num_features, 64),
#                 nn.ReLU(),
#                 nn.Linear(64, 1)
#             ) for _ in range(out_dim)
#         ])

#     def forward(self, x):
#         base_output = self.base_model(x)
#         features = base_output

#         # Apply each MLP head to the base model output
#         outputs = [head(features) for head in self.mlp_heads]

#         # Concatenate the outputs along the last dimension
#         return torch.cat(outputs, dim=1)

def create_model(student_model_name, teacher_model_name, pretrained, num_classes, in_chans, out_dim, load_from_file=False, model_path=None):
    teacher_model = TeacherModel(model_name=teacher_model_name, pretrained=pretrained, num_classes=num_classes, load_from_file=load_from_file, model_path=model_path)
    student_model = StudentModel(model_name=student_model_name, pretrained=pretrained, num_classes=num_classes, in_chans=in_chans)
    model = DistillationModel(teacher_model, student_model)

    return model


In [9]:
import torch

model_name = 'swin_tiny_patch4_window7_224'
teacher_model_name = 'resnet50'
pretrained = True
num_classes = 0
in_chans = 3
out_dim = 5

tmodel = TeacherModel(model_name=teacher_model_name, pretrained=pretrained, num_classes=num_classes)

# 랜덤 입력 생성 (배치 크기 1, 채널 수 3, 이미지 크기 224x224)
random_input = torch.randn(1, in_chans, 224, 224)

# Pass the input through the model
student_output_resized, teacher_output, distillation_loss = model(random_input)  # Unpack all three values

# Check the shape of the output
print("Output shape:", output.shape)

# Optionally print the feature maps from intermediate layers
for i, feature in enumerate(features):
    print(f"Feature {i+1} shape:", feature.shape)

# print(model)

After patch embedding: torch.Size([1, 56, 56, 96])
torch.Size([1, 7, 7, 768])
torch.Size([1, 7, 7, 768])
before global pool: torch.Size([1, 7, 7, 768])
after: torch.Size([1, 768])
mlp input: torch.Size([1, 768])


ValueError: not enough values to unpack (expected 3, got 1)

In [10]:
import torch

model_name = 'swin_tiny_patch4_window7_224'
teacher_model_name = 'resnet50'
pretrained = True
num_classes = 0
in_chans = 3
out_dim = 5

smodel = StudentModel(model_name=model_name, pretrained=pretrained, num_classes=num_classes, in_chans=in_chans)

# 랜덤 입력 생성 (배치 크기 1, 채널 수 3, 이미지 크기 224x224)
random_input = torch.randn(1, in_chans, 224, 224)

# Pass the input through the model
student_output_resized, teacher_output, distillation_loss = model(random_input)  # Unpack all three values

# Check the shape of the output
print("Output shape:", output.shape)
# print(features)
# Optionally print the feature maps from intermediate layers
for i, feature in enumerate(features):
    print(f"Feature {i+1} shape:", feature.shape)
# print(model.base_model)
# print(model)

After patch embedding: torch.Size([1, 56, 56, 96])
torch.Size([1, 7, 7, 768])
torch.Size([1, 7, 7, 768])
before global pool: torch.Size([1, 7, 7, 768])
after: torch.Size([1, 768])
mlp input: torch.Size([1, 768])


ValueError: not enough values to unpack (expected 3, got 1)

In [45]:
import torch
import timm
from torch import nn
import torch.nn.functional as F

# 먼저 TeacherModel, StudentModel, FeatureTransform, DistillationModel 클래스가 
# 위에서 제공한 수정된 버전으로 정의되어 있는지 확인해주세요.

model_name = 'swin_tiny_patch4_window7_224'
teacher_model_name = 'resnet50'
pretrained = True
num_classes = 0
in_chans = 3
out_dim = 5

# 모델 새로 생성
teacher_model = TeacherModel(model_name=teacher_model_name, pretrained=pretrained, num_classes=num_classes)
student_model = StudentModel(model_name=model_name, pretrained=pretrained, num_classes=num_classes, in_chans=in_chans)
model = DistillationModel(teacher_model=teacher_model, student_model=student_model)

# 랜덤 입력 생성
random_input = torch.randn(1, in_chans, 224, 224)

# 모델에 입력 통과
student_output_resized, teacher_output, distillation_loss = model(random_input)

# 출력 shape와 distillation loss 확인
print("Student Output shape:", student_output_resized.shape)
print("Teacher Output shape:", teacher_output.shape)
print(f"Distillation loss: {distillation_loss}")

# # 중간 특징 맵 출력 및 변환 과정 확인
# for i, (transform, student_feature, teacher_feature) in enumerate(zip(model.transforms, model.student_model.features, model.teacher_model.features)):
#     print(f"\nLayer {i+1}:")
#     print(f"Student Feature original shape: {student_feature.shape}")
    
#     # Student 특징 맵을 [B, C, H, W] 형태로 변환
#     student_feature_permuted = student_feature.permute(0, 3, 1, 2)
#     print(f"Student Feature permuted shape: {student_feature_permuted.shape}")
    
#     # 변환된 특징 맵
#     transformed_feature = transform(student_feature_permuted)
#     print(f"Student Feature transformed shape: {transformed_feature.shape}")
    
#     print(f"Teacher Feature shape: {teacher_feature.shape}")
    
#     # Shape 일치 여부 확인
#     if transformed_feature.shape == teacher_feature.shape:
#         print("Shapes match!")
#     else:
#         print("Shapes do not match.")

ValueError: too many values to unpack (expected 3)

In [39]:
import torch

model_name = 'swin_tiny_patch4_window7_224'
teacher_model_name = 'resnet50'
pretrained = True
num_classes = 0
in_chans = 3
out_dim = 5

# Create distillation model
model = create_model(model_name, teacher_model_name, pretrained, num_classes, in_chans, out_dim)
model.to('cpu')

# Generate random input (batch size = 1, 3 channels, 224x224 image)
random_input = torch.randn(1, in_chans, 224, 224)

# Pass the input through the model
student_output_resized, teacher_output, distillation_loss = model(random_input)

# Check the shape of the output and distillation loss
print("Student Output shape:", student_output_resized.shape)
print("Teacher Output shape:", teacher_output.shape)
print(f"Total Distillation loss: {distillation_loss.item()}")

Student Output shape: torch.Size([1, 5])
Teacher Output shape: torch.Size([1, 2048])
Total Distillation loss: 22.66550064086914


In [34]:
import torch

# 모델을 생성
model_name = 'swin_tiny_patch4_window7_224'
pretrained = True
num_classes = 0
in_chans = 3
out_dim = 5

# 모델 생성
model = create_model(model_name, pretrained, num_classes, in_chans, out_dim)

# 랜덤 입력 생성 (배치 크기 1, 채널 수 3, 이미지 크기 224x224)
random_input = torch.randn(1, in_chans, 224, 224)

# 모델에 랜덤 입력을 넣어 출력 확인
output = model(random_input)

print("Output shape:", output.shape)  # 출력의 크기 확인


TypeError: create_model() missing 1 required positional argument: 'out_dim'

In [35]:
model = timm.create_model(model_name='swin_s3_base_224', pretrained=True, num_classes=0, in_chans=3)
print(model)

SwinTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
  )
  (layers): Sequential(
    (0): SwinTransformerStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): SwinTransformerBlock(
          (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (attn): WindowAttention(
            (qkv): Linear(in_features=96, out_features=288, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=96, out_features=96, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (drop_path1): Identity()
          (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=96, out_features=384, bias=True)
            (act): GELU(approximate='none')
            (drop1): 

In [36]:
import torch

# 테스트를 위한 설정 값
student_model_name = "swin_tiny_patch4_window7_224"  # 예: Swin-Tiny
teacher_model_name = "resnet50"  # 예: ResNet50
pretrained = False
num_classes = 5
in_chans = 3
out_dim = 5  # 최종 출력 차원 (예: 5개의 클래스를 예측한다고 가정)
load_from_file = False
model_path = None

# 임의의 입력 데이터 생성 (배치 크기 2, 채널 3, 높이 224, 너비 224)
input_tensor = torch.randn(2, in_chans, 224, 224)

# 모델 생성
model = create_model(
    student_model_name=student_model_name,
    teacher_model_name=teacher_model_name,
    pretrained=pretrained,
    num_classes=num_classes,
    in_chans=in_chans,
    out_dim=out_dim,
    load_from_file=load_from_file,
    model_path=model_path
)

# DistillationModel의 forward 테스트
model.eval()  # 평가 모드로 설정 (gradient 업데이트 방지)
with torch.no_grad():
    student_output_resized, teacher_output, distillation_loss = model(input_tensor)

# 결과 출력
print("Student Output Resized:", student_output_resized.shape)
print("Teacher Output:", teacher_output.shape)
print("Distillation Loss:", distillation_loss.item())

# TeacherModel 및 StudentModel의 feature 확인
print("\n--- Feature Shapes ---")
print("Teacher Features:")
for i, feat in enumerate(model.teacher_model.features):
    print(f"  Layer {i+1}: {feat.shape}")

print("Student Features:")
for i, feat in enumerate(model.student_model.features):
    print(f"  Layer {i+1}: {feat.shape}")


RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x5 and 768x5)

In [31]:
import torch
model_name = 'swin_tiny_patch4_window7_224'
teacher_model_name = 'resnet50'
pretrained = True
num_classes = 5
in_chans = 3
out_dim = 5

# Create distillation model
model = create_model(model_name, teacher_model_name, pretrained, num_classes, in_chans, out_dim)
model.to('cpu')

# Generate random input (batch size = 1, 3 channels, 224x224 image)
random_input = torch.randn(1, in_chans, 224, 224)

# Pass the input through the model
student_output_resized, teacher_output, distillation_loss = model(random_input)

# Check the shape of the output and distillation loss
print("Student Output shape:", student_output_resized.shape)
print("Teacher Output shape:", teacher_output.shape)
print(f"Distillation loss: {distillation_loss}")

# Print the feature maps from intermediate layers
for i, (transform, student_feature, teacher_feature) in enumerate(zip(model.transforms, model.student_model.features, model.teacher_model.features)):
    print(f"Student Feature {i+1} original shape:", student_feature.shape)
    transformed_feature = transform(student_feature.permute(0, 3, 1, 2))
    print(f"Student Feature {i+1} transformed shape:", transformed_feature.shape)
    print(f"Teacher Feature {i+1} shape:", teacher_feature.shape)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x5 and 768x5)

In [41]:
import torch

# Model 설정
student_model_name = 'swin_tiny_patch4_window7_224'
teacher_model_name = 'resnet50'
pretrained = True
num_classes = 0
in_chans = 3
out_dim = 5

# Distillation model 생성
model = create_model(student_model_name, teacher_model_name, pretrained, num_classes, in_chans, out_dim)
model.to('cpu')

# 임의의 입력 데이터 생성 (batch size = 1, 3 channels, 224x224 image)
random_input = torch.randn(1, in_chans, 224, 224)

# forward pass를 통해 레이어별 feature shape 확인
with torch.no_grad():
    _, _, _ = model(random_input)

# 각 레이어의 feature shape 출력
print("\n--- Teacher Model Feature Shapes ---")
for i, feat in enumerate(model.teacher_model.features):
    print(f"Teacher Layer {i+1}: {feat.shape}")

print("\n--- Student Model Feature Shapes ---")
for i, feat in enumerate(model.student_model.features):
    print(f"Student Layer {i+1}: {feat.shape}")



--- Teacher Model Feature Shapes ---
Teacher Layer 1: torch.Size([1, 256, 56, 56])
Teacher Layer 2: torch.Size([1, 512, 28, 28])
Teacher Layer 3: torch.Size([1, 1024, 14, 14])

--- Student Model Feature Shapes ---
Student Layer 1: torch.Size([1, 56, 56, 96])
Student Layer 2: torch.Size([1, 28, 28, 192])
Student Layer 3: torch.Size([1, 14, 14, 384])
Student Layer 4: torch.Size([1, 7, 7, 768])


In [44]:
import torch

# Model 설정
student_model_name = 'swin_tiny_patch4_window7_224'
teacher_model_name = 'resnet50'
pretrained = True
num_classes = 0
in_chans = 3
out_dim = 5
beta = 1.0

# Distillation model 생성
teacher_model = TeacherModel(model_name=teacher_model_name, pretrained=pretrained, num_classes=num_classes)
student_model = StudentModel(model_name=student_model_name, pretrained=pretrained, num_classes=num_classes, in_chans=in_chans)
model = DistillationModel(teacher_model, student_model, beta=beta)
model.to('cpu')

# 임의의 입력 데이터 생성 (batch size = 1, 3 channels, 224x224 image)
random_input = torch.randn(1, in_chans, 224, 224)

# forward pass를 통해 레이어별 feature shape 확인
with torch.no_grad():
    student_output_resized, teacher_output, _ = model(random_input)

# 각 레이어의 feature shape 출력
print("\n--- Teacher Model Feature Shapes ---")
for i, feat in enumerate(model.teacher_model.features):
    print(f"Teacher Layer {i+1}: {feat.shape}")

print("\n--- Student Model Feature Shapes (Before Transformation) ---")
for i, feat in enumerate(model.student_model.features):
    print(f"Student Layer {i+1}: {feat.shape}")

# 변환 후 feature shape 확인
print("\n--- Transformed Student Feature Shapes (After Alignment) ---")
for i, (transform, s_feat, t_feat) in enumerate(zip(model.transforms, model.student_model.features[:-1], model.teacher_model.features)):
    # 채널 변환 및 공간 정렬 수행
    transformed_feat = F.interpolate(transform(s_feat), size=(t_feat.shape[2], t_feat.shape[3]), mode='bilinear')
    print(f"Transformed Student Layer {i+1}: {transformed_feat.shape} (aligned to Teacher Layer {i+1})")


ValueError: too many values to unpack (expected 3)

In [32]:
from functools import partial
from itertools import repeat
import collections.abc as container_abcs

import logging
import os
from collections import OrderedDict

import numpy as np
import scipy
import torch
import torch.nn as nn
import torch.nn.functional as F
from einops import rearrange
from einops.layers.torch import Rearrange

from timm.models.layers import DropPath, trunc_normal_

#from .registry import register_model


# From PyTorch internals
def _ntuple(n):
    def parse(x):
        if isinstance(x, container_abcs.Iterable):
            return x
        return tuple(repeat(x, n))

    return parse


to_1tuple = _ntuple(1)
to_2tuple = _ntuple(2)
to_3tuple = _ntuple(3)
to_4tuple = _ntuple(4)
to_ntuple = _ntuple


class LayerNorm(nn.LayerNorm):
    """Subclass torch's LayerNorm to handle fp16."""

    def forward(self, x: torch.Tensor):
        orig_type = x.dtype
        ret = super().forward(x.type(torch.float32))
        return ret.type(orig_type)


class QuickGELU(nn.Module):
    def forward(self, x: torch.Tensor):
        return x * torch.sigmoid(1.702 * x)


class Mlp(nn.Module):
    def __init__(self,
                 in_features,
                 hidden_features=None,
                 out_features=None,
                 act_layer=nn.GELU,
                 drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x


class Attention(nn.Module):
    def __init__(self,
                 dim_in,
                 dim_out,
                 num_heads,
                 qkv_bias=False,
                 attn_drop=0.,
                 proj_drop=0.,
                 method='dw_bn',
                 kernel_size=3,
                 stride_kv=1,
                 stride_q=1,
                 padding_kv=1,
                 padding_q=1,
                 with_cls_token=True,
                 **kwargs
                 ):
        super().__init__()
        self.stride_kv = stride_kv
        self.stride_q = stride_q
        self.dim = dim_out
        self.num_heads = num_heads
        # head_dim = self.qkv_dim // num_heads
        self.scale = dim_out ** -0.5
        self.with_cls_token = with_cls_token

        self.conv_proj_q = self._build_projection(
            dim_in, dim_out, kernel_size, padding_q,
            stride_q, 'linear' if method == 'avg' else method
        )
        self.conv_proj_k = self._build_projection(
            dim_in, dim_out, kernel_size, padding_kv,
            stride_kv, method
        )
        self.conv_proj_v = self._build_projection(
            dim_in, dim_out, kernel_size, padding_kv,
            stride_kv, method
        )

        self.proj_q = nn.Linear(dim_in, dim_out, bias=qkv_bias)
        self.proj_k = nn.Linear(dim_in, dim_out, bias=qkv_bias)
        self.proj_v = nn.Linear(dim_in, dim_out, bias=qkv_bias)

        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim_out, dim_out)
        self.proj_drop = nn.Dropout(proj_drop)

    def _build_projection(self,
                          dim_in,
                          dim_out,
                          kernel_size,
                          padding,
                          stride,
                          method):
        if method == 'dw_bn':
            proj = nn.Sequential(OrderedDict([
                ('conv', nn.Conv2d(
                    dim_in,
                    dim_in,
                    kernel_size=kernel_size,
                    padding=padding,
                    stride=stride,
                    bias=False,
                    groups=dim_in
                )),
                ('bn', nn.BatchNorm2d(dim_in)),
                ('rearrage', Rearrange('b c h w -> b (h w) c')),
            ]))
        elif method == 'avg':
            proj = nn.Sequential(OrderedDict([
                ('avg', nn.AvgPool2d(
                    kernel_size=kernel_size,
                    padding=padding,
                    stride=stride,
                    ceil_mode=True
                )),
                ('rearrage', Rearrange('b c h w -> b (h w) c')),
            ]))
        elif method == 'linear':
            proj = None
        else:
            raise ValueError('Unknown method ({})'.format(method))

        return proj

    def forward_conv(self, x, h, w):
        if self.with_cls_token:
            cls_token, x = torch.split(x, [1, h*w], 1)

        x = rearrange(x, 'b (h w) c -> b c h w', h=h, w=w)

        if self.conv_proj_q is not None:
            q = self.conv_proj_q(x)
        else:
            q = rearrange(x, 'b c h w -> b (h w) c')

        if self.conv_proj_k is not None:
            k = self.conv_proj_k(x)
        else:
            k = rearrange(x, 'b c h w -> b (h w) c')

        if self.conv_proj_v is not None:
            v = self.conv_proj_v(x)
        else:
            v = rearrange(x, 'b c h w -> b (h w) c')

        if self.with_cls_token:
            q = torch.cat((cls_token, q), dim=1)
            k = torch.cat((cls_token, k), dim=1)
            v = torch.cat((cls_token, v), dim=1)

        return q, k, v

    def forward(self, x, h, w):
        if (
            self.conv_proj_q is not None
            or self.conv_proj_k is not None
            or self.conv_proj_v is not None
        ):
            q, k, v = self.forward_conv(x, h, w)

        q = rearrange(self.proj_q(q), 'b t (h d) -> b h t d', h=self.num_heads)
        k = rearrange(self.proj_k(k), 'b t (h d) -> b h t d', h=self.num_heads)
        v = rearrange(self.proj_v(v), 'b t (h d) -> b h t d', h=self.num_heads)

        attn_score = torch.einsum('bhlk,bhtk->bhlt', [q, k]) * self.scale
        attn = F.softmax(attn_score, dim=-1)
        attn = self.attn_drop(attn)

        x = torch.einsum('bhlt,bhtv->bhlv', [attn, v])
        x = rearrange(x, 'b h t d -> b t (h d)')

        x = self.proj(x)
        x = self.proj_drop(x)

        return x

    @staticmethod
    def compute_macs(module, input, output):
        # T: num_token
        # S: num_token
        input = input[0]
        flops = 0

        _, T, C = input.shape
        H = W = int(np.sqrt(T-1)) if module.with_cls_token else int(np.sqrt(T))

        H_Q = H / module.stride_q
        W_Q = H / module.stride_q
        T_Q = H_Q * W_Q + 1 if module.with_cls_token else H_Q * W_Q

        H_KV = H / module.stride_kv
        W_KV = W / module.stride_kv
        T_KV = H_KV * W_KV + 1 if module.with_cls_token else H_KV * W_KV

        # C = module.dim
        # S = T
        # Scaled-dot-product macs
        # [B x T x C] x [B x C x T] --> [B x T x S]
        # multiplication-addition is counted as 1 because operations can be fused
        flops += T_Q * T_KV * module.dim
        # [B x T x S] x [B x S x C] --> [B x T x C]
        flops += T_Q * module.dim * T_KV

        if (
            hasattr(module, 'conv_proj_q')
            and hasattr(module.conv_proj_q, 'conv')
        ):
            params = sum(
                [
                    p.numel()
                    for p in module.conv_proj_q.conv.parameters()
                ]
            )
            flops += params * H_Q * W_Q

        if (
            hasattr(module, 'conv_proj_k')
            and hasattr(module.conv_proj_k, 'conv')
        ):
            params = sum(
                [
                    p.numel()
                    for p in module.conv_proj_k.conv.parameters()
                ]
            )
            flops += params * H_KV * W_KV

        if (
            hasattr(module, 'conv_proj_v')
            and hasattr(module.conv_proj_v, 'conv')
        ):
            params = sum(
                [
                    p.numel()
                    for p in module.conv_proj_v.conv.parameters()
                ]
            )
            flops += params * H_KV * W_KV

        params = sum([p.numel() for p in module.proj_q.parameters()])
        flops += params * T_Q
        params = sum([p.numel() for p in module.proj_k.parameters()])
        flops += params * T_KV
        params = sum([p.numel() for p in module.proj_v.parameters()])
        flops += params * T_KV
        params = sum([p.numel() for p in module.proj.parameters()])
        flops += params * T

        module.__flops__ += flops


class Block(nn.Module):

    def __init__(self,
                 dim_in,
                 dim_out,
                 num_heads,
                 mlp_ratio=4.,
                 qkv_bias=False,
                 drop=0.,
                 attn_drop=0.,
                 drop_path=0.,
                 act_layer=nn.GELU,
                 norm_layer=nn.LayerNorm,
                 **kwargs):
        super().__init__()

        self.with_cls_token = kwargs['with_cls_token']

        self.norm1 = norm_layer(dim_in)
        self.attn = Attention(
            dim_in, dim_out, num_heads, qkv_bias, attn_drop, drop,
            **kwargs
        )

        self.drop_path = DropPath(drop_path) \
            if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim_out)

        dim_mlp_hidden = int(dim_out * mlp_ratio)
        self.mlp = Mlp(
            in_features=dim_out,
            hidden_features=dim_mlp_hidden,
            act_layer=act_layer,
            drop=drop
        )

    def forward(self, x, h, w):
        res = x

        x = self.norm1(x)
        attn = self.attn(x, h, w)
        x = res + self.drop_path(attn)
        x = x + self.drop_path(self.mlp(self.norm2(x)))

        return x


class ConvEmbed(nn.Module):
    """ Image to Conv Embedding

    """

    def __init__(self,
                 patch_size=7,
                 in_chans=3,
                 embed_dim=64,
                 stride=4,
                 padding=2,
                 norm_layer=None):
        super().__init__()
        patch_size = to_2tuple(patch_size)
        self.patch_size = patch_size

        self.proj = nn.Conv2d(
            in_chans, embed_dim,
            kernel_size=patch_size,
            stride=stride,
            padding=padding
        )
        self.norm = norm_layer(embed_dim) if norm_layer else None

    def forward(self, x):
        x = self.proj(x)

        B, C, H, W = x.shape
        x = rearrange(x, 'b c h w -> b (h w) c')
        if self.norm:
            x = self.norm(x)
        x = rearrange(x, 'b (h w) c -> b c h w', h=H, w=W)

        return x


class VisionTransformer(nn.Module):
    """ Vision Transformer with support for patch or hybrid CNN input stage
    """
    def __init__(self,
                 patch_size=16,
                 patch_stride=16,
                 patch_padding=0,
                 in_chans=3,
                 embed_dim=768,
                 depth=12,
                 num_heads=12,
                 mlp_ratio=4.,
                 qkv_bias=False,
                 drop_rate=0.,
                 attn_drop_rate=0.,
                 drop_path_rate=0.,
                 act_layer=nn.GELU,
                 norm_layer=nn.LayerNorm,
                 init='trunc_norm',
                 **kwargs):
        super().__init__()
        self.num_features = self.embed_dim = embed_dim  # num_features for consistency with other models

        self.rearrage = None

        self.patch_embed = ConvEmbed(
            # img_size=img_size,
            patch_size=patch_size,
            in_chans=in_chans,
            stride=patch_stride,
            padding=patch_padding,
            embed_dim=embed_dim,
            norm_layer=norm_layer
        )

        with_cls_token = kwargs['with_cls_token']
        if with_cls_token:
            self.cls_token = nn.Parameter(
                torch.zeros(1, 1, embed_dim)
            )
        else:
            self.cls_token = None

        self.pos_drop = nn.Dropout(p=drop_rate)
        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, depth)]  # stochastic depth decay rule

        blocks = []
        for j in range(depth):
            blocks.append(
                Block(
                    dim_in=embed_dim,
                    dim_out=embed_dim,
                    num_heads=num_heads,
                    mlp_ratio=mlp_ratio,
                    qkv_bias=qkv_bias,
                    drop=drop_rate,
                    attn_drop=attn_drop_rate,
                    drop_path=dpr[j],
                    act_layer=act_layer,
                    norm_layer=norm_layer,
                    **kwargs
                )
            )
        self.blocks = nn.ModuleList(blocks)

        if self.cls_token is not None:
            trunc_normal_(self.cls_token, std=.02)

        if init == 'xavier':
            self.apply(self._init_weights_xavier)
        else:
            self.apply(self._init_weights_trunc_normal)

    def _init_weights_trunc_normal(self, m):
        if isinstance(m, nn.Linear):
            logging.info('=> init weight of Linear from trunc norm')
            trunc_normal_(m.weight, std=0.02)
            if m.bias is not None:
                logging.info('=> init bias of Linear to zeros')
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, (nn.LayerNorm, nn.BatchNorm2d)):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def _init_weights_xavier(self, m):
        if isinstance(m, nn.Linear):
            logging.info('=> init weight of Linear from xavier uniform')
            nn.init.xavier_uniform_(m.weight)
            if m.bias is not None:
                logging.info('=> init bias of Linear to zeros')
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, (nn.LayerNorm, nn.BatchNorm2d)):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def forward(self, x):
        x = self.patch_embed(x)
        B, C, H, W = x.size()

        x = rearrange(x, 'b c h w -> b (h w) c')

        cls_tokens = None
        if self.cls_token is not None:
            # stole cls_tokens impl from Phil Wang, thanks
            cls_tokens = self.cls_token.expand(B, -1, -1)
            x = torch.cat((cls_tokens, x), dim=1)

        x = self.pos_drop(x)

        for i, blk in enumerate(self.blocks):
            x = blk(x, H, W)

        if self.cls_token is not None:
            cls_tokens, x = torch.split(x, [1, H*W], 1)
        x = rearrange(x, 'b (h w) c -> b c h w', h=H, w=W)

        return x, cls_tokens


class ConvolutionalVisionTransformer(nn.Module):
    def __init__(self,
                 in_chans=3,
                 num_classes=1000,
                 act_layer=nn.GELU,
                 norm_layer=nn.LayerNorm,
                 init='trunc_norm',
                 spec=None):
        super().__init__()
        self.num_classes = num_classes

        self.num_stages = spec['NUM_STAGES']
        for i in range(self.num_stages):
            kwargs = {
                'patch_size': spec['PATCH_SIZE'][i],
                'patch_stride': spec['PATCH_STRIDE'][i],
                'patch_padding': spec['PATCH_PADDING'][i],
                'embed_dim': spec['DIM_EMBED'][i],
                'depth': spec['DEPTH'][i],
                'num_heads': spec['NUM_HEADS'][i],
                'mlp_ratio': spec['MLP_RATIO'][i],
                'qkv_bias': spec['QKV_BIAS'][i],
                'drop_rate': spec['DROP_RATE'][i],
                'attn_drop_rate': spec['ATTN_DROP_RATE'][i],
                'drop_path_rate': spec['DROP_PATH_RATE'][i],
                'with_cls_token': spec['CLS_TOKEN'][i],
                'method': spec['QKV_PROJ_METHOD'][i],
                'kernel_size': spec['KERNEL_QKV'][i],
                'padding_q': spec['PADDING_Q'][i],
                'padding_kv': spec['PADDING_KV'][i],
                'stride_kv': spec['STRIDE_KV'][i],
                'stride_q': spec['STRIDE_Q'][i],
            }

            stage = VisionTransformer(
                in_chans=in_chans,
                init=init,
                act_layer=act_layer,
                norm_layer=norm_layer,
                **kwargs
            )
            setattr(self, f'stage{i}', stage)

            in_chans = spec['DIM_EMBED'][i]

        dim_embed = spec['DIM_EMBED'][-1]
        self.norm = norm_layer(dim_embed)
        self.num_features = dim_embed
        self.cls_token = spec['CLS_TOKEN'][-1]

        # Classifier head
        self.head = nn.Linear(dim_embed, num_classes) if num_classes > 0 else nn.Identity()
        trunc_normal_(self.head.weight, std=0.02)

    def init_weights(self, pretrained='', pretrained_layers=[], verbose=True):
        if os.path.isfile(pretrained):
            pretrained_dict = torch.load(pretrained, map_location='cpu')
            logging.info(f'=> loading pretrained model {pretrained}')
            model_dict = self.state_dict()
            pretrained_dict = {
                k: v for k, v in pretrained_dict.items()
                if k in model_dict.keys()
            }
            need_init_state_dict = {}
            for k, v in pretrained_dict.items():
                need_init = (
                        k.split('.')[0] in pretrained_layers
                        or pretrained_layers[0] == '*'
                )
                if need_init:
                    if verbose:
                        logging.info(f'=> init {k} from {pretrained}')
                    if 'pos_embed' in k and v.size() != model_dict[k].size():
                        size_pretrained = v.size()
                        size_new = model_dict[k].size()
                        logging.info(
                            '=> load_pretrained: resized variant: {} to {}'
                            .format(size_pretrained, size_new)
                        )

                        ntok_new = size_new[1]
                        ntok_new -= 1

                        posemb_tok, posemb_grid = v[:, :1], v[0, 1:]

                        gs_old = int(np.sqrt(len(posemb_grid)))
                        gs_new = int(np.sqrt(ntok_new))

                        logging.info(
                            '=> load_pretrained: grid-size from {} to {}'
                            .format(gs_old, gs_new)
                        )

                        posemb_grid = posemb_grid.reshape(gs_old, gs_old, -1)
                        zoom = (gs_new / gs_old, gs_new / gs_old, 1)
                        posemb_grid = scipy.ndimage.zoom(
                            posemb_grid, zoom, order=1
                        )
                        posemb_grid = posemb_grid.reshape(1, gs_new ** 2, -1)
                        v = torch.tensor(
                            np.concatenate([posemb_tok, posemb_grid], axis=1)
                        )

                    need_init_state_dict[k] = v
            self.load_state_dict(need_init_state_dict, strict=False)

    @torch.jit.ignore
    def no_weight_decay(self):
        layers = set()
        for i in range(self.num_stages):
            layers.add(f'stage{i}.pos_embed')
            layers.add(f'stage{i}.cls_token')

        return layers

    def forward_features(self, x):
        for i in range(self.num_stages):
            x, cls_tokens = getattr(self, f'stage{i}')(x)

        if self.cls_token:
            x = self.norm(cls_tokens)
            x = torch.squeeze(x)
        else:
            x = rearrange(x, 'b c h w -> b (h w) c')
            x = self.norm(x)
            x = torch.mean(x, dim=1)

        return x

    def forward(self, x):
        x = self.forward_features(x)
        x = self.head(x)

        return x

In [36]:
import torch
import torch.nn as nn
from functools import partial
from collections import namedtuple

# Provided CvT model definition imports
# Assume that ConvolutionalVisionTransformer and LayerNorm are defined as per the code you provided

# Create a configuration for the CvT model
Config = namedtuple('Config', ['MODEL', 'VERBOSE'])
ModelConfig = namedtuple('ModelConfig', ['SPEC', 'NUM_CLASSES', 'PRETRAINED', 'PRETRAINED_LAYERS', 'INIT_WEIGHTS'])
SpecConfig = namedtuple('SpecConfig', [
    'NUM_STAGES', 'PATCH_SIZE', 'PATCH_STRIDE', 'PATCH_PADDING', 'DIM_EMBED', 'DEPTH',
    'NUM_HEADS', 'MLP_RATIO', 'QKV_BIAS', 'DROP_RATE', 'ATTN_DROP_RATE', 'DROP_PATH_RATE',
    'CLS_TOKEN', 'QKV_PROJ_METHOD', 'KERNEL_QKV', 'PADDING_Q', 'PADDING_KV', 'STRIDE_KV', 'STRIDE_Q'
])

# Define the model specifications
spec = SpecConfig(
    NUM_STAGES=3,
    PATCH_SIZE=[7, 3, 3],
    PATCH_STRIDE=[4, 2, 2],
    PATCH_PADDING=[2, 1, 1],
    DIM_EMBED=[64, 192, 384],
    DEPTH=[1, 2, 10],
    NUM_HEADS=[1, 3, 6],
    MLP_RATIO=[4.0, 4.0, 4.0],
    QKV_BIAS=[True, True, True],
    DROP_RATE=[0.0, 0.0, 0.0],
    ATTN_DROP_RATE=[0.0, 0.0, 0.0],
    DROP_PATH_RATE=[0.0, 0.1, 0.2],
    CLS_TOKEN=[False, False, True],
    QKV_PROJ_METHOD=['dw_bn', 'dw_bn', 'dw_bn'],
    KERNEL_QKV=[3, 3, 3],
    PADDING_Q=[1, 1, 1],
    PADDING_KV=[1, 1, 1],
    STRIDE_KV=[2, 2, 2],
    STRIDE_Q=[1, 1, 1]
)

# Define the model configuration
model_config = ModelConfig(
    SPEC=spec,
    NUM_CLASSES=1000,
    PRETRAINED='',
    PRETRAINED_LAYERS=['*'],
    INIT_WEIGHTS=True
)

# Create the full configuration
config = Config(
    MODEL=model_config,
    VERBOSE=True
)

# Instantiate the CvT model
cvt_model = ConvolutionalVisionTransformer(
    in_chans=3,
    num_classes=config.MODEL.NUM_CLASSES,
    act_layer=QuickGELU,
    norm_layer=partial(LayerNorm, eps=1e-5),
    init='trunc_norm',
    spec=config.MODEL.SPEC._asdict()
)

# Initialize model weights if required
if config.MODEL.INIT_WEIGHTS:
    cvt_model.init_weights(
        config.MODEL.PRETRAINED,
        config.MODEL.PRETRAINED_LAYERS,
        config.VERBOSE
    )

# Create a random input tensor with the shape (batch_size, channels, height, width)
input_tensor = torch.randn(1, 3, 224, 224)

# Set the model to evaluation mode
cvt_model.eval()

# Perform a forward pass
with torch.no_grad():
    output = cvt_model(input_tensor)

# Print the output shape (should be [batch_size, num_classes])
print("Output shape:", output.shape) 

class CvtModel(nn.Module):
    def __init__(self, cvt_model):
        super(CvtModel, self).__init__()
        self.base_model = cvt_model

        self.base_model.head = nn.Identity()
        
        self.avg_pool = nn.AvgPool2d(kernel_size=2, stride=2)
        
    def forward(self, x):
        print(x.shape)
        
        x, _ = self.base_model.stage0(x)
        #print(x.shape, cls_tokens)
        
        x, _ = self.base_model.stage1(x)
        #print(x.shape, cls_tokens)
        
        x, cls_tokens = self.base_model.stage2(x)
        print(x.shape, cls_tokens.shape)
        
        x = self.avg_pool(x) # 14x14에서 7x7로 다운샘플링
        print("after avg pool:", x.shape)
        
        cls_tokens = self.base_model.norm(cls_tokens)
        cls_tokens = torch.squeeze(cls_tokens, dim=1)
        print(cls_tokens.shape)
        
        return cls_tokens

class MLP_layer(nn.Module):
    def __init__(self, base_model, out_dim):
        super().__init__()
        self.base_model = base_model
        self.out_dim = out_dim

        self.num_features = self.base_model.base_model.num_features
        
        # out_dim 개수만큼 MLP 생성
        self.mlp_heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(self.num_features, 64),
                nn.ReLU(),
                nn.Linear(64, 1)
            ) for _ in range(out_dim)
        ])

    def forward(self, x):
        base_output = self.base_model(x)
        
        print(base_output.shape)
        
        features = base_output

        outputs = [head(features) for head in self.mlp_heads]

        return torch.cat(outputs, dim=1)

model = CvtModel(cvt_model)
model = MLP_layer(model, 5)

with torch.no_grad():
    output = model(input_tensor)

# Print the output shape (should be [batch_size, num_classes])
print("Output shape:", output.shape)

Output shape: torch.Size([1000])
torch.Size([1, 3, 224, 224])
torch.Size([1, 384, 14, 14]) torch.Size([1, 1, 384])
after avg pool: torch.Size([1, 384, 7, 7])
torch.Size([1, 384])
torch.Size([1, 384])
Output shape: torch.Size([1, 5])


In [16]:
import timm
import torch
import torch.nn as nn
import sys
import os

# sys.path.append('/home/work/')
# print(sys.path)


from T2T_ViT.T2T_ViT_main.models.t2t_vit import t2t_vit_14

class BaseModel(nn.Module):
    def __init__(self, model_name, pretrained, num_classes, in_chans):
        super(BaseModel, self).__init__()
        self.base_model = t2t_vit_14()
        self.base_model.head = nn.Identity()
        
        self.avg_pool = nn.AvgPool2d(kernel_size=2, stride=2)
        
    def forward(self, x):
        B = x.shape[0]
        
        x = self.base_model.tokens_to_token(x)
        print(x.shape)
        
        cls_tokens = self.base_model.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        print("After adding cls_token:", x.shape)
        
        x = x + self.base_model.pos_embed
        x = self.base_model.pos_drop(x)
        print(x.shape)
        
        for i, block in enumerate(self.base_model.blocks) :
            x = block(x)
        
        x = self.base_model.norm(x)
        print(x.shape)
        
        cls_token = x[:, 0]
        print("cls_token:", cls_token.shape)
        
        patch_token = x[:, 1:] # (batch_size, 196, 768)
        batch_size, num_patches, embed_dim = patch_token.shape
        patch_token = patch_token.view(batch_size, 14, 14, embed_dim) # n, k, k, d
        patch_token = patch_token.permute(0, 3, 1, 2) # n, d, k, k
        patch_token = self.avg_pool(patch_token) # n, d, 14, 14 -> n, d, 7, 7
        
        print("patch_token:", patch_token.shape)
        
        
        return cls_token

    

class MLP_layer(nn.Module):
    def __init__(self, base_model, out_dim):
        super().__init__()
        self.base_model = base_model
        self.out_dim = out_dim

        self.num_features = self.base_model.base_model.num_features
        
        # out_dim 개수만큼 MLP 생성
        self.mlp_heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(self.num_features, 64),
                nn.ReLU(),
                nn.Linear(64, 1)
            ) for _ in range(out_dim)
        ])

    def forward(self, x):
        base_output = self.base_model(x)
        
        features = base_output

        outputs = [head(features) for head in self.mlp_heads]

        return torch.cat(outputs, dim=1)

def create_model(model_name, pretrained, num_classes, in_chans, out_dim):

    if out_dim <= 0:
        raise ValueError("오류: out_dim이 0 이하입니다.")

    base_model = BaseModel(model_name, pretrained, num_classes, in_chans)
    model = MLP_layer(base_model, out_dim)

    return model


model = create_model('', False, 0, 3, 5)

input_tensor = torch.randn(1, 3, 224, 224)

# Set the model to evaluation mode
model.eval()

# Perform a forward pass
with torch.no_grad():
    output = model(input_tensor)

# Print the output shape (should be [batch_size, num_classes])
print("Output shape:", output.shape)

adopt performer encoder for tokens-to-token
torch.Size([1, 196, 384])
After adding cls_token: torch.Size([1, 197, 384])
torch.Size([1, 197, 384])
torch.Size([1, 197, 384])
cls_token: torch.Size([1, 384])
patch_token: torch.Size([1, 384, 7, 7])
Output shape: torch.Size([1, 5])


In [2]:
import timm
import torch
import torch.nn as nn
import mlflow
import mlflow.pyfunc

class BaseModel(nn.Module):
    def __init__(self, model_name, pretrained, num_classes, in_chans):
        super(BaseModel, self).__init__()
        self.base_model = timm.create_model(model_name=model_name, pretrained=pretrained, num_classes=num_classes, in_chans=in_chans)

    def forward(self, x):
        x = self.base_model(x)
        return x

    

class MLP_layer(nn.Module):
    def __init__(self, base_model, out_dim):
        super().__init__()
        self.base_model = base_model
        self.out_dim = out_dim

        # Freeze base_model
        for param in self.base_model.parameters():
            param.requires_grad = False

        self.num_features = self.base_model.num_features
        
        # out_dim 개수만큼 MLP 생성
        self.mlp_heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(self.num_features, 64),
                nn.ReLU(),
                nn.Linear(64, 1)
            ) for _ in range(out_dim)
        ])

    def forward(self, x):
        base_output = self.base_model(x)
        
        features = base_output

        outputs = [head(features) for head in self.mlp_heads]

        return torch.cat(outputs, dim=1)

def create_model(model_name, pretrained, num_classes, in_chans, out_dim):

    if out_dim <= 0:
        raise ValueError("오류: out_dim이 0 이하입니다.")

    mlflow.set_tracking_uri("http://0.0.0.0:5000")

    model_name = "Classification Model"  # 등록된 모델 이름
    model_version = 1  # 버전 번호

    model_uri = f"models:/{model_name}/{model_version}"

    base_model = mlflow.pytorch.load_model(model_uri)

    base_model.head = nn.Identity()

    model = MLP_layer(base_model, out_dim)

    return model



In [3]:
model_name = 'vit_base_r50_s16_224.orig_in21k'
pretrained = True
num_classes = 0
in_chans = 3
out_dim = 5

model = create_model(model_name, pretrained, num_classes, in_chans, out_dim)

In [4]:
print(model)

MLP_layer(
  (base_model): VisionTransformer(
    (patch_embed): HybridEmbed(
      (backbone): ResNetV2(
        (stem): Sequential(
          (conv): StdConv2dSame(3, 64, kernel_size=(7, 7), stride=(2, 2), bias=False)
          (norm): GroupNormAct(
            32, 64, eps=1e-05, affine=True
            (drop): Identity()
            (act): ReLU(inplace=True)
          )
          (pool): MaxPool2dSame(kernel_size=(3, 3), stride=(2, 2), padding=(0, 0), dilation=(1, 1), ceil_mode=False)
        )
        (stages): Sequential(
          (0): ResNetStage(
            (blocks): Sequential(
              (0): Bottleneck(
                (downsample): DownsampleConv(
                  (conv): StdConv2dSame(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
                  (norm): GroupNormAct(
                    32, 256, eps=1e-05, affine=True
                    (drop): Identity()
                    (act): Identity()
                  )
                )
                (conv1): 

In [11]:
import timm
import torch
import torch.nn as nn

class SelectAdaptivePool2d(nn.Module):
    def __init__(self, pool_type='avg', flatten=True):
        super(SelectAdaptivePool2d, self).__init__()
        if pool_type == 'avg':
            self.pool = nn.AdaptiveAvgPool2d((1, 1))  # Adaptive Average Pooling
        elif pool_type == 'max':
            self.pool = nn.AdaptiveMaxPool2d((1, 1))  # Adaptive Max Pooling
        else:
            raise ValueError("Unsupported pool_type: {}".format(pool_type))
        
        self.flatten = flatten

    def forward(self, x):
        # x shape: (batch_size, 7, 7, 768)
        x = x.permute(0, 3, 1, 2)  # Change to (batch_size, 768, 7, 7)
        x = self.pool(x)  # Pooling to (batch_size, 768, 1, 1)
        if self.flatten:
            x = x.view(x.size(0), -1)  # Flatten to (batch_size, 768)
        return x 

class BaseModel(nn.Module):
    def __init__(self, model_name, pretrained, num_classes, in_chans):
        super(BaseModel, self).__init__()
        self.base_model = timm.create_model(model_name=model_name, pretrained=pretrained, num_classes=num_classes, in_chans=in_chans)

        self.base_model.head.global_pool = nn.Identity()
        self.global_pool = SelectAdaptivePool2d()

    def forward(self, x, m):
        
        print(x.shape)
        
        x = self.base_model(x) # x: n, k, k, d
        
        print(x.shape)
        
        patch_token = x.permute(0, 3, 1, 2) # n, d, k, k
        
        print(patch_token.shape)
        
        # drloc_loss 계산
        #drloc_loss = dense_relative_localization_loss(patch_token, drloc_mlp, m)

        
        print(x.shape)
        # Classifier head 적용
        x = self.global_pool(x)
        
        print(x.shape)
        
        return x#, drloc_loss

In [12]:
model_name = 'swin_tiny_patch4_window7_224'
pretrained = True
num_classes = 0
in_chans = 3
base_model = BaseModel(model_name, pretrained, num_classes, in_chans)

In [13]:
input_tensor = torch.randn(1, 3, 224, 224)

# Set the model to evaluation mode
base_model.eval()

# Perform a forward pass
with torch.no_grad():
    output = base_model(input_tensor, m=64)

torch.Size([1, 3, 224, 224])
torch.Size([1, 7, 7, 768])
torch.Size([1, 768, 7, 7])
torch.Size([1, 7, 7, 768])
torch.Size([1, 768])
